In [1]:
from main import *
from tqdm import tqdm

In [2]:
domain = 'pollini'
model_path = "data/chkpt/"+domain+"/0001.pt"

config_path = "config/" + domain
config = Config()
config.load_from_file(config_path)
TRAJ_MAX_LEN=99
graph, trajectories, pairwise_node_features, _ = load_data(config)
model = create_model(graph, pairwise_node_features, config)
model = model.to(config.device)
model.load_state_dict(torch.load(model_path)["model_state_dict"])
model.eval()
graph = graph.to(config.device)

0 0

1 19

2 45

3 90

4 115

5 91

6 0

7 20

8 0

9 19

10 45

11 67

12 0

13 19

14 45

15 68

16 91

17 0

18 21

19 68

20 45

21 69

22 139

23 157

24 115

25 0

26 19

27 46

28 171

29 0

30 20

31 1

32 45

33 90

34 115

35 92

36 45

37 67

38 0

39 19

40 45

41 68

42 47

43 20

44 1

45 45

46 68

47 46

48 115

49 91

50 0

51 19

52 45

53 0

54 19

55 45

56 68

57 115

58 91

59 0

60 19

61 48

62 45

63 90

64 116

65 0

66 19

67 45

68 68

69 20

70 1

71 45

72 68

73 49

74 140

75 140

76 141

77 184

78 115

79 91

80 0

81 19

82 45

83 91

84 0

85 22

86 184

87 202

88 0

89 19

90 45

91 19

92 45

93 67

94 0

95 0

96 19

97 45

98 20

99 2

100 115

101 219

102 172

103 45

104 67

105 3

106 172

107 0

108 20

109 1

110 45

111 68

112 47

113 0

114 20

115 91

116 0

117 23

118 91

119 90

120 116

121 0

122 20

123 47

124 0

125 19

126 45

127 68

128 46

129 115

130 90

131 115

132 91

133 0

134 19

135 45

136 0

137 24

138 115

139 

AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [4]:
TRAJ_MAX_LEN=99

In [5]:
def create_evaluator():
    return Evaluator(graph.n_node, given_as_target=None, siblings_nodes=None,config=config,)
def predict(model, graph, trajectories, config, pairwise_features=None):
    preds= []
    with torch.no_grad():
        for trajectory_idx in tqdm(range(len(trajectories))):
            observations = trajectories[trajectory_idx]
            number_steps = None
            if config.rw_edge_weight_see_number_step or config.rw_expected_steps:
                if config.use_shortest_path_distance:
                    number_steps = (
                        trajectories.leg_shortest_lengths(
                            trajectory_idx).float() * 1.1
                    ).long()
                else:
                    number_steps = trajectories.leg_lengths(trajectory_idx)

            observed, starts, targets = generate_masks(
                trajectory_length=observations.shape[0],
                number_observations=config.number_observations,
                predict=config.target_prediction,
                with_interpolation=config.with_interpolation,
                device=config.device,
            )

            diffusion_graph = (
                graph if not config.diffusion_self_loops else graph.add_self_loops()
            )

            predictions, _, rw_weights = model(
                observations,
                graph,
                diffusion_graph,
                observed=observed,
                starts=starts,
                targets=targets,
                pairwise_node_features=pairwise_features,
                number_steps=number_steps,
            )
            preds.append(predictions.argmax(dim=1))
        
    return preds

In [6]:
valid_trajectories = trajectories[2].to(config.device)
evaluate(model, graph,
        valid_trajectories, pairwise_node_features,
        create_evaluator, dataset="EVAL")

  0%|          | 0/3402 [00:00<?, ?it/s]


=== EVAL ===



/home/vikramreddyardham/code/gretel-path-extrapolation/metrics.py:238: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554786529/work/aten/src/ATen/native/IndexingUtils.h:30.)
  traversed_edges = traversed_edges[~duplicate_mask]
100%|██████████| 3402/3402 [05:29<00:00, 10.31it/s]

| Metric               |     Value |
|----------------------|-----------|
| target_probability   |  0.260704 |
| choice_accuracy      |  0.157756 |
| choice_accuracy_deg3 |  0.157743 |
| precision_top1       |  0.357834 |
| precision_top5       |  0.84447  |
| path_nll             | 27.3691   |
| path_nll_deg3        | 27.3688   |


In [7]:
edges2ids = {(val[0].item(), val[1].item()):idd for idd, val in enumerate(zip(graph.senders,graph.receivers))}

In [16]:
TRAJ_MAX_LEN=100
def path2traj(paths, num_nodes):
    """converts a simple nodewise list of paths to trajectory format required by gretel"""
    
    if not isinstance(paths[0],list):
        """if there is just one path in traj"""
        paths=[paths]
    
    total_observations = sum([len(path) for path in paths])
    lengths=[]
    indices = torch.zeros(total_observations, 1, dtype=torch.long)
    num_observations = 0
    
    for path in paths:
        length = len(path)
        #assert length > config.min_trajectory_length, "Min length constraint not satisfied"
        lengths.append(length) 
        for i,node in enumerate(path):
            indices[i+num_observations, 0] = node
        num_observations+=length  
    
    weights = torch.ones(num_observations, 1)
    
    num_paths = num_observations - len(paths)
    max_path_length= TRAJ_MAX_LEN
    trails = torch.zeros([num_paths, max_path_length], dtype=torch.long) - 1
    for i, path in enumerate(paths):
        edges = [edges2ids[(e1,e2)] for (e1,e2) in zip(path[:-1],path[1:])]
        trails[i, :len(edges)] = torch.tensor(edges)
    return Trajectories(weights, indices, num_nodes, torch.tensor(lengths), trails)

In [17]:
paths = [[1,3,0,1,0, 2,0,4, 5, 6, 5, 1], [0, 1, 3,3, 0, 0, 1, 0 ,5, 6, 5,1]]
trajs = path2traj(paths, 47)
trajs = trajs.to('cuda:0')

NameError: name 'edges2ids' is not defined

In [25]:
preds = predict(model, graph, trajs, config)

100%|██████████| 2/2 [00:00<00:00, 186.47it/s]


In [2]:
import torch

In [3]:
temp = torch.tensor([[1, float('inf'), float('nan')]])

In [7]:
torch.isfinite(temp).flatten().sum()

tensor(1)